In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gym
from collections import deque
from time import sleep
#import pickle 
#file_pi = open('DQN_Memory.obj', 'wb') 


gamma = 0.95
epsilon_max= 1
epsilon_rate = 0.9995
epsilon_min = 0.1
batch_size = 32
memory_replay_size = 100000
# self.AGENT_HISTORY_LENGTH = 4
target_update_length = 200
alpha = 0.00025
#   self.GRADIENT_MOMENTUM = 0.95
#  self.SQUARED_GRADIENT_MOMENTUM = 0.95
# self.MIN_SQUARED_GRADIENT = 0.01
replay_start_size = 9000
memory = deque(maxlen = 10000)      # ***********************          save and load self.MEMORY       *****************************************
#self.MODEL = self.CNN_initialize()
#self.TARGET_MODEL = self.CNN_initialize()
steps = 0

env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n




class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,20,3)
        self.conv3 = nn.Conv2d(20,30,3)
        self.fc1 = nn.Linear( 30*5*5 ,400)
        self.fc2 = nn.Linear(400,64)
        self.fc3 = nn.Linear(64,10)
        self.softmax = nn.Softmax(dim = 1)
    
    def forward(self,x):
        x = F.max_pool2d(torch.relu(self.conv2(torch.relu(self.conv1(x)))),(2,2))
        x = F.max_pool2d(torch.relu(self.conv3(x)),(2,2))
        x = x.view(1,self.num_of_features(x))
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x
    
    
    def train(self,X, Y, epochs , batch_size , learning_rate ):
        optimizer = optim.SGD(self.parameters(), lr = learning_rate)
        criterion = nn.SmoothL1Loss()
        accuracy = 0.0
        for epoch in range(1,epochs+1,1):
            if epoch>1:
                print('--------------->>>>>>   Epoch {} with accuracy {}'.format(epoch, loss))
            permutation = torch.randperm(X.shape[1])
            for i in range(0, X.shape[1] - batch_size, batch_size):
                optimizer.zero_grad()
                indices = permutation[i:i+batch_size]
                batchx, batchy = X[indices], Y[indices]
                outputs = self.forward(torch.from_numpy(batchx).view(batch_size,1,28,28).float())
                loss = criterion(outputs, torch.from_numpy(batchy).view(batch_size,10).float())
                loss.backward()
                optimizer.step()
        return
    
    def evaluat(self, X):
        #self.eval()
        with torch.no_grad():
            for i in range(X.shape[0]):
                output = net.forward(torch.from_numpy(X[i]).view(1,1,28,28).float())
                plt.subplot(1+X.shape[0]//3,3,i+1)
                plt.imshow(np.reshape(X[i],(28,28)))
                plt.title('GIVEN : {}'.format(output))
        return
            
    
    def num_of_features(self,x):
        features = x.size()[1:]
        num = 1
        for feature in features:
            num *= feature
        return num




In [ ]:
class game:
  def __init__(self,state_size,action_size):
    self.state_size=state_size
    self.action_size=action_size
    self.GAMMA = 0.95
    self.EPSILON_MAX = 1
    self.EPSILON_RATE = 0.9995
    self.EPSILON_MIN = 0.1
    self.MINIBATCH_SIZE = 32
    self.MEMORY_REPLAY_SIZE = 100000
    self.AGENT_HISTORY_LENGTH = 4
    self.TARGET_UPDATE_LENGTH = 200
    self.ALPHA = 0.00025
    self.GRADIENT_MOMENTUM = 0.95
    self.SQUARED_GRADIENT_MOMENTUM = 0.95
    self.MIN_SQUARED_GRADIENT = 0.01
    self.REPLAY_START_SIZE = 9000
    self.MEMORY = deque(maxlen = 10000)      # ***********************          save and load self.MEMORY       *****************************************
    self.MODEL = self.CNN_initialize()
    self.TARGET_MODEL = self.CNN_initialize()
    self.STEPS = 0
    #Incomplete
  
  
  def target_update(self):
    self.TARGET_MODEL.set_weights(self.MODEL.get_weights())
  

  def CNN_initialize(self):
    model = keras.models.Sequential()
    # model.add(keras.layers.Conv2D(32,(8,8), strides = (4,4), input_shape = (167,81,1),padding = 'valid'))
    # model.add(keras.layers.Activation('relu'))
    # model.add(keras.layers.Conv2D(64,(4,4),strides = (2,2), padding = 'valid'))
    # model.add(keras.layers.Activation('relu'))
    # model.add(keras.layers.Conv2D(64,(3,3),strides = (1,1), padding = 'valid'))
    # model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512,activation = 'relu'))
    model.add(keras.layers.Dense(200,activation='relu'))
    model.add(keras.layers.Dense(2,kernel_initializer=keras.initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='normal', seed=None),activation='linear'))
    model.compile(loss = 'mean_squared_error',optimizer= keras.optimizers.Adam(0.0001), metrics=["accuracy"])
    return model
    # MODEL HERE


  def preprocess(self,arr):  
    out_arr = arr[0]
    out_arr = out_arr[30:197,:]
    out_arr = np.delete(out_arr, 2*np.arange(79)+1, axis = 1)
    out_arr = out_arr/np.max(out_arr)
    out_arr = np.reshape(out_arr, (1,167,81,1))
    return out_arr

  def preprocess_out(self,arr):  
    out_arr = arr[1]
    out_arr = out_arr[30:197,:]
    out_arr = np.delete(out_arr, 2*np.arange(79)+1, axis = 1)
    out_arr = out_arr/np.max(out_arr)
    out_arr = np.reshape(out_arr, (1,167,81,1))
    return out_arr
  
  def act(self,state,epsilon):
    #if np.random.rand() <= epsilon:
     # return np.random.choice(self.action_size)
    act_values=self.MODEL.predict(state)
    return np.argmax(act_values[0])
  
  def save(self):
    self.MODEL.save_weights('CARTPOLE_MODEL_WEIGHTS_2.h5')                 # ******************  SPECIFY PATH  *********************************
    self.TARGET_MODEL.save_weights('CARTPOLE_MODEL_WEIGHTS_2.h5')
    print('Model Saved')
  
  def load(self):
    self.MODEL.set_weights('CARTPOLE_MODEL_WEIGHTS.h5')       # ********************** SPECIFY PATH ****************************
    self.target_update()
    print('Target Loaded')
       # UPDATE    CSV   self.Memory
  

  def remember(self,array):
    self.MEMORY.append(array)
    #print(len(self.MEMORY))
    self.STEPS += 1
    if self.STEPS % 2000 == 0:
      #pickle.dump(self.MEMORY, file_pi)
      self.target_update()
      #self.save()
      print(str(self.STEPS)+ ' steps completed')

In [ ]:
env.reset()
agent = game(state_size,action_size)
agent.target_update()
print('Training Started')
agent.load()
epsilon = agent.EPSILON_MAX
for N in range(20000):
  env.reset()
  print('Episode Started')
  present_img = deque([],maxlen = 2)
  tot_reward = 0
  y = np.array([0,0])
  tot_reward = 0
  present_img.append(env.reset())
  T =0
  done = False
  while done == False:
    T += 1
    #if T%100==0:
      #print(str(T)+' TIME STEPS DONE')
      #print(tot_reward)
      #print(len(agent.MEMORY))
      # *******************************COMPUTE Q OUTPUT ******************************
    done = False                                                                             #next_state,reward,done,info
    train_img = present_img[0]
    Action = agent.act(train_img,epsilon)

    

    out = env.step(Action)
    env.render()
    #sleep(0.005)
    tot_reward += out[1]
    done = out[2]
    present_img.append(out[0])

    next_img = present_img[1] #***************************************
    agent.remember((train_img,out[1],Action, next_img,done))

      

    if len(agent.MEMORY) > agent.REPLAY_START_SIZE:
      #print('Updating Model')
      epsilon = max(epsilon,agent.EPSILON_MIN)
      epsilon = epsilon * agent.EPSILON_RATE
      indices = np.random.choice(np.arange(0,len(agent.MEMORY)-1,1),32)
      batch = [agent.MEMORY[i] for i in indices]
      for img_input,Reward,action,target_input,over in batch:
        y = agent.MODEL.predict(img_input)
        if over:
          y[0][action] = Reward - np.abs(img_input[2])/50
        else:
          y[0][action] = Reward - np.abs(img_input[2])/50 + agent.GAMMA * np.amax(agent.TARGET_MODEL.predict(target_input)[0])
        agent.MODEL.fit(img_input,y,epochs = 1, verbose = 0)  


    if done:
      print("Episode {}/{} finished after {} timesteps epsilon:{} total reward:{} total steps:{}".format(N,20000,T+1,epsilon,tot_reward,agent.STEPS))

env.close()